In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [58]:
url_inicial = 'https://www.rmangiologia.com/?indice=2020484#JournalContents'
url_root = 'https://www.rmangiologia.com/?indice=2020484#JournalContents'
r=requests.get(url_inicial)

In [59]:
soup = BeautifulSoup(r.text, 'html.parser')

In [60]:
#box = soup.find('div', class_='container')
volumen=soup.findAll('div', class_="item")
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.rmangiologia.com/frame_esp.php?id=42',
 'https://www.rmangiologia.com/frame_esp.php?id=43',
 'https://www.rmangiologia.com/frame_esp.php?id=44',
 'https://www.rmangiologia.com/frame_esp.php?id=45',
 'https://www.rmangiologia.com/frame_esp.php?id=46']

In [61]:
def get_url_items(sopa,url):
    #box = soup.find('div', class_='container')
    volumen=soup.findAll('div', class_="item")
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [62]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.rmangiologia.com/?indice=2020484#JournalContents


In [63]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

5

In [64]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [65]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Angiología'
    a='III. Medicina y Ciencias de la Salud'
    tem='Medicina'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('h4', class_='callout-title').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('div', class_='col-lg-2 col-md-2 col-sm-2').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [66]:
list_scraper=list_scraper[0:6]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4


In [67]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Angiología,III. Medicina y Ciencias de la Salud,Medicina,Tratamiento farmacomecánico dirigido por catét...,None,None,https://www.rmangiologia.com/frame_esp.php?id=42,https://www.rmangiologia.com/files/rma_20_48_4...
1,Revista Mexicana de Angiología,III. Medicina y Ciencias de la Salud,Medicina,Utilidad del índice tobillo-brazo en el trauma...,None,None,https://www.rmangiologia.com/frame_esp.php?id=43,https://www.rmangiologia.com/files/rma_20_48_4...
2,Revista Mexicana de Angiología,III. Medicina y Ciencias de la Salud,Medicina,Angiographic predictors of post-operative atri...,None,None,https://www.rmangiologia.com/frame_esp.php?id=44,https://www.rmangiologia.com/files/rma_20_48_4...
3,Revista Mexicana de Angiología,III. Medicina y Ciencias de la Salud,Medicina,Asociación entre la distancia a base de cráneo...,None,None,https://www.rmangiologia.com/frame_esp.php?id=45,https://www.rmangiologia.com/files/rma_20_48_4...
4,Revista Mexicana de Angiología,III. Medicina y Ciencias de la Salud,Medicina,Tratamiento quirúrgico del traumatismo vascula...,None,None,https://www.rmangiologia.com/frame_esp.php?id=46,https://www.rmangiologia.com/files/rma_20_48_4...


In [57]:
df_catalogo.to_csv('Revista099.07.csv', index=False)